In [1]:
# -- ------------------------------------------------------------- Importar modulos a utilizar -- #
# -- ------------------------------------------------------------- --------------------------- -- #

import ta as ta
import pandas as pd
from oandapyV20 import API
import oandapyV20.endpoints.instruments as instruments
import plotly.graph_objs as go
import plotly as py

In [2]:
# -- -------------------------------------------------------------- Parametros para Data Frame -- #
# -- ------------------------------------------------------------------- --------------------- -- #

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 200)
pd.set_option('display.width', 1000)

In [15]:
# -- ------------------------------------------------------------------- Parametros para OANDA -- #
# -- ------------------------------------------------------------------- --------------------- -- #

A1_OA_Da = 16                     # Day Align
A1_OA_Ta = "America/Mexico_City"  # Time Align

A1_OA_Ai = "101-004-2221697-001"  # Id de cuenta
A1_OA_At = "practice"             # Tipo de cuenta

A1_OA_In = "USD_MXN"              # Instrumento
A1_OA_Gn = "M5"                   # Granularidad de velas

A1_OA_Ak = '2' + '6b62ddbe404c61a1cc9da0ed8395945-52d44ef76c42b62460581783bba6c8e' + 'a'

F1 = "2019-03-01T00:00:00Z"
F2 = "2017-03-01T10:00:00Z"

In [ ]:
# -- -------------------------------------------------------------------- Estructuras de datos -- #
# -- ------------------------------------------------------------------- --------------------- -- #

# -- Paso 1 -- Data Frame (df1_precios) descargando todos los precios, segun fechas y granularidad
# -- -- Fecha, Apertura, Maximo, Minimo, Cierre.

# -- Paso 2 -- en Data Frame (df2_operaciones)
# -- -- Fecha, Folio (1 a n), Operacion (1 = compra, -1 = venta), Unidades (0 a n),
# -- -- Margen (0 a n), Comentario ("razon con base al indicador")
# -- -- Precio_apertura, Precio_cierre

# -- Paso 3 -- en Data Frame Cuenta (df3_cuenta)
# -- -- Fecha, Capital (Efectivo $), Flotante ($),
# -- -- Balance (Capital+Flotante), Rend_Balance_acm, Comentario ("Se abrio operacion: compra/venta",
# -- -- "Se cerro operacion: Con perdia de: / Con ganancia de: ", 

# -- Paso 4 -- Generar un vector de fechas, de F1 hasta F2

par0_cap = 100000   # Capital a utilizar $100,000 Usd
par1_mar = 1000000  # Unidades por operacion maximo 1'000,000


# -- Criterios para trading
# -- para cerrar una operacion: 1) TP o SL, 2) Regla con indicador

indice = [1, 2, 3, 4, 5]
columnas = ['A','B', 'C', 'D']
df = pd.DataFrame(index=indice, columns=columnas)
df = df.fillna(0) # -- llenar todas las celdas con 0s

In [16]:
# -- ---------------------------------------------------------------- Inicializar API de OANDA -- #
# -- ---------------------------------------------------------------- ------------------------ -- #

api = API(access_token=A1_OA_Ak)

In [17]:
# -- -------------------------------------------------------------- Obtener precios historicos -- #
# -- -------------------------------------------------------------- -------------------------- -- #

params = {"granularity": A1_OA_Gn, "price": "M", "dailyAlignment": A1_OA_Da,
          "alignmentTimezone": A1_OA_Ta, "from": F1, "to": F2}

A1_Req1 = instruments.InstrumentsCandles(instrument=A1_OA_In, params=params)
A1_Hist = api.request(A1_Req1)

In [18]:
lista = []
for i in range(len(A1_Hist['candles'])-1):
        lista.append({'TimeStamp': A1_Hist['candles'][i]['time'],
                      'Open': A1_Hist['candles'][i]['mid']['o'],
                      'High': A1_Hist['candles'][i]['mid']['h'],
                      'Low': A1_Hist['candles'][i]['mid']['l'],
                      'Close': A1_Hist['candles'][i]['mid']['c']})

pd_hist = pd.DataFrame(lista)
pd_hist = pd_hist[['TimeStamp', 'Open', 'High', 'Low', 'Close']]
pd_hist['TimeStamp'] = pd.to_datetime(pd_hist['TimeStamp'])

In [19]:
pd_hist

,TimeStamp,Open,High,Low,Close
0,2017-01-02 23:00:00,20.73596,20.75360,20.73468,20.74616
1,2017-01-03 00:00:00,20.74619,20.74727,20.74033,20.74154
2,2017-01-03 01:00:00,20.74155,20.74554,20.74062,20.74499
3,2017-01-03 02:00:00,20.74500,20.74532,20.72468,20.72468
4,2017-01-03 03:00:00,20.72466,20.72492,20.71882,20.72060
...,...,...,...,...,...
125,2017-01-10 04:00:00,21.39858,21.40048,21.39064,21.39229
126,2017-01-10 05:00:00,21.39230,21.41634,21.39195,21.40652
127,2017-01-10 06:00:00,21.40644,21.40832,21.39704,21.40043
128,2017-01-10 07:00:00,21.40044,21.41188,21.38409,21.39438


In [12]:
# -- ---------------------------------------------------------------- Agregar analisis tecnico -- #
# -- ---------------------------------------------------------------- ------------------------ -- #

pd_hist['ema_Close'] = ta.ema(series=pd_hist['Close'], periods=20)

In [13]:
# -- ------------------------------------------------------------------- --------------------- -- #
# -- ------------------------------------------------------------------- --------------------- -- #

trace0 = go.Scatter(x=pd_hist['TimeStamp'], y=pd_hist['ema_Close'],
                    mode='lines', name='lines0')

trace1 = go.Scatter(x=pd_hist['TimeStamp'], y=pd_hist['Close'],
                    mode='lines', name='lines1')

data = [trace0, trace1]

py.offline.plot(data, filename='file.html')

'file:///media/franciscome/DATA/GitHub/MyST/Notas_Python/Notas_AnalisisTecnico/file.html'